In [190]:
import pandas as pd
from itertools import combinations

def interaction_matrix(df):
    """takes a pandas dataframe of main effects as input, and 
    calculates and concatenates all higher order interaction effects"""
    main_effects = list(df.columns)
    num_main_effects = len(main_effects) 
    for i in range(2, num_main_effects+1):
        combos = list(combinations(main_effects, i))
        for effect in combos:
            interaction = df[effect[0]]
            name = effect[0]
            for j in range(1,i):
                name += effect[j]
                interaction = interaction * df[effect[j]]
                interaction = interaction.rename(name)
            df = pd.concat([df, interaction],axis=1)
    return df


def getDuplicateColumns(df):
    """takes an interaction matrix (pandas dataframe) as input, 
    and returns a list of all confounding variables. That is,
    all variables with exactly duplicated settings"""
    duplicateColumnNames = set()
    for x in range(df.shape[1]):
        col = df.iloc[:, x]
        for y in range(x + 1, df.shape[1]):
            otherCol = df.iloc[:, y]
            if col.equals(otherCol):
                duplicateColumnNames.add(df.columns.values[y])
    return list(duplicateColumnNames)


def confounding_with_x(factor, interaction_matrix):
    """takes a factor name (stringcolumn name in df) and an 
    interaction matrix (pandas dataframe) as input, and 
    returns a list of confounding variables for that factor"""
    x = interaction_matrix.loc[:, factor]
    confound_x = []
    for column in interaction_matrix:
        if interaction_matrix[column].equals(interaction_matrix[factor]):
            confound_x.append(column)
    result = 'Effects confounded with ' + factor + ': ' + str(confound_x)
    return result

In [191]:
if __name__ == "__main__":
    A = [-1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1, -1, 1]
    B = [-1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1, -1, -1, 1, 1]
    C = [-1, -1, -1, -1, 1, 1, 1, 1, -1, -1, -1, -1, 1, 1, 1, 1]
    D = [-1, -1, -1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1]
    E = [-1, 1, 1, -1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, -1, 1]
    F = [-1, -1, 1, 1, 1, 1, -1, -1, 1, 1, -1, -1, -1, -1, 1, 1]

    dict = {'A':A, 'B':B, 'C':C, 'D':D, 'E':E, 'F':F}
    df = pd.DataFrame(dict)
    print('MAIN EFFECTS WITH GENERATORS E=ABC AND F=BCD')
    display(df)
    print('\n')
    
    interaction_matrix = interaction_matrix(df)
    print('INTERACTION MATRIX')
    display(interaction_matrix)
    print('\n')
    
    print('ALL CONFOUNDING EFFECTS')
    print(getDuplicateColumns(interaction_matrix))
    print('\n')
    
    print(confounding_with_x('C',interaction_matrix))
    print('\n')
    
    print(confounding_with_x('AB',interaction_matrix))
    print('\n')
    
    print(confounding_with_x('CD',interaction_matrix))

MAIN EFFECTS WITH GENERATORS E=ABC AND F=BCD


,A,B,C,D,E,F
0,-1,-1,-1,-1,-1,-1
1,1,-1,-1,-1,1,-1
2,-1,1,-1,-1,1,1
3,1,1,-1,-1,-1,1
4,-1,-1,1,-1,1,1
5,1,-1,1,-1,-1,1
6,-1,1,1,-1,-1,-1
7,1,1,1,-1,1,-1
8,-1,-1,-1,1,-1,1
9,1,-1,-1,1,1,1




INTERACTION MATRIX


,A,B,C,D,E,F,AB,AC,AD,AE,...,BCEF,BDEF,CDEF,ABCDE,ABCDF,ABCEF,ABDEF,ACDEF,BCDEF,ABCDEF
0,-1,-1,-1,-1,-1,-1,1,1,1,1,...,1,1,1,-1,-1,-1,-1,-1,-1,1
1,1,-1,-1,-1,1,-1,-1,-1,-1,1,...,-1,-1,-1,-1,1,-1,-1,-1,1,1
2,-1,1,-1,-1,1,1,-1,1,1,-1,...,-1,-1,1,-1,-1,1,1,-1,1,-1
3,1,1,-1,-1,-1,1,1,-1,-1,-1,...,1,1,-1,-1,1,1,1,-1,-1,-1
4,-1,-1,1,-1,1,1,1,-1,1,-1,...,-1,1,-1,-1,-1,1,-1,1,1,-1
5,1,-1,1,-1,-1,1,-1,1,-1,-1,...,1,-1,1,-1,1,1,-1,1,-1,-1
6,-1,1,1,-1,-1,-1,-1,-1,1,1,...,1,-1,-1,-1,-1,-1,1,1,-1,1
7,1,1,1,-1,1,-1,1,1,-1,1,...,-1,1,1,-1,1,-1,1,1,1,1
8,-1,-1,-1,1,-1,1,1,1,-1,1,...,-1,1,1,1,-1,1,-1,-1,-1,1
9,1,-1,-1,1,1,1,-1,-1,1,1,...,1,-1,-1,1,1,1,-1,-1,1,1




ALL CONFOUNDING EFFECTS
['BCD', 'BDE', 'ADEF', 'BCDE', 'ACF', 'ABC', 'ADE', 'BEF', 'DF', 'BCDEF', 'ABE', 'BCEF', 'AEF', 'ACDF', 'CE', 'CDF', 'BCE', 'BDEF', 'BCF', 'CF', 'ABCD', 'ACDEF', 'BCDF', 'BE', 'ABDF', 'ABCDF', 'DEF', 'ACE', 'ABEF', 'ABDEF', 'CDE', 'ACD', 'DE', 'ACEF', 'ABCEF', 'ABCDEF', 'CDEF', 'CD', 'ADF', 'ACDE', 'ABCDE', 'EF', 'ABCF', 'BDF', 'CEF', 'ABDE', 'BC']


Effects confounded with C: ['C', 'ABE', 'BDF', 'ACDEF']


Effects confounded with AB: ['AB', 'CE', 'ACDF', 'BDEF']


Effects confounded with CD: ['BF', 'CD', 'ABDE', 'ACEF']
